# DAP on simulation results for calcium pathway

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
# from pandasql import sqldf
from scipy import stats
# import pickle
import feather
from collections import Counter
from datetime import datetime
from utils import run_dap, load_data
import os
filename = "data/calcium_pathway_4000_shape_25.data.blocks.pkl"
fileout = "data/calcium_pathway_4000_shape_25.data"
# print(fileout)

## Data transform: $y \in \{0,1\}$ to $y \in \{0,t\}$

In [2]:
data = load_data(filename)
# the lifetime prevalence of schizophrenia is 4.0/1000, not 1%, but we first set it as 0.01
# according to https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1140960/
# relative risk is normally between 2 and 10
RR = 2.5
prevalence = data['debug']['args']['prevalence']
t = stats.norm(0,1).ppf(1-prevalence)
# effect size of beta is z_mu, prior
z_mu = t - stats.norm(0,1).ppf(1-RR*prevalence)
print (t, z_mu)

2.57582930355 0.334426575944


In [3]:
# there are 0~1847 blocks, block #776 has the most genes involved
# i = 0
# for dat in data["data"]:
#     print (len(dat.columns.values[1:].tolist()), i)
#     i+=1

In [4]:
# create prior file
cal_pthwy_genes = pd.read_table("../data/calciumgeneset.txt", skiprows = 2, header = None, names = ["gene_name"])
ref_genes = pd.read_table("../data/refGene.txt.gz", compression="gzip", sep="\t", header = None, 
                          usecols=(2,12), names=["chrom", "gene_name"]).drop_duplicates(subset=("gene_name"))
calpath_gene_pos = pd.merge(ref_genes, cal_pthwy_genes, how="inner", on=["gene_name"])
diff = set(cal_pthwy_genes["gene_name"].tolist()) - set(calpath_gene_pos["gene_name"].tolist())
# add the two missing genes and their chromosome in the dataframe
calpath_gene_pos.loc[calpath_gene_pos.shape[0]] = ["chr2", list(diff)[0]]
calpath_gene_pos.loc[calpath_gene_pos.shape[0]] = ["chr15", list(diff)[1]]

In [5]:
calpath_gene_pos["chr.gene"] = calpath_gene_pos.apply(lambda row: "{}.{}".format(row["chrom"], row["gene_name"]), 
                                                      axis=1)
calpath_gene_pos["prior.pr"] = calpath_gene_pos.apply(lambda row: np.random.uniform(0, z_mu), axis=1)

In [6]:
beta = z_mu
y_tran = t

## Run TORUS

In [7]:
# from get_sum_stats_for_torus import get_summary_stats
import fisher

In [27]:
def get_sum_stats_cal_simu(gene_df, causal, cat, multi_cat = 0, subst = -10, chrom = "chr6", seed = 999):
    '''get summary statistics (e.g. t-statistic, p-value) to be used for TORUS, then transfer p-value to z-score
       Actually do NOT need SNP (gene) map and gene (chromosome) map
       If multi_cat!=0, then cat must be multiple categories
       If multi_cat==0, then cat can be any list (because it won't be used) but must be given
    '''
    np.random.seed(seed)
    cases = gene_df[gene_df["phenotype"]==1]
    ctrls = gene_df[gene_df["phenotype"]==0]
    df_cal_bf = pd.DataFrame()
    df_cal_beta = pd.DataFrame()
    df_cal_zscore = pd.DataFrame()
    SNP_anno_cal = pd.DataFrame()
    i = 1
    for gene in list(gene_df)[1:]:
        n_gene_case = len(cases[cases[gene]==1])
        n_nogene_case = len(cases[cases[gene]==0])
        n_gene_ctrl = len(ctrls[ctrls[gene]==1])
        n_nogene_ctrl = len(ctrls[ctrls[gene]==0])
        if n_gene_case == 0 or n_gene_ctrl == 0:
            odds_ratio = 0
        else:
            odds_ratio = n_gene_case * n_nogene_ctrl / n_gene_ctrl / n_nogene_case
        print (n_gene_case, n_nogene_case, n_gene_ctrl, n_nogene_ctrl, odds_ratio)
        if np.isinf(odds_ratio):
            beta1 = 10
        elif odds_ratio == 0:
            beta1 = -10
        else:
            beta1 = np.log(odds_ratio)
        fisher_stats = fisher.pvalue(n_gene_case, n_gene_ctrl, n_nogene_case, n_nogene_ctrl)
        p_value_two = fisher_stats.two_tail
        p_value_left = fisher_stats.left_tail
        p_value_right = fisher_stats.right_tail
        if p_value_right < 0.5 and p_value_two < 1:
            z_score = stats.norm.ppf(1-p_value_right)
        elif p_value_left < 0.5 and p_value_two < 1:
            z_score = -stats.norm.ppf(1-p_value_left)
        else:
            z_score = subst
        df_cal_bf = df_cal_bf.append([("{}.{}".format(chrom, gene), chrom, odds_ratio)])
        df_cal_beta = df_cal_beta.append([("{}.{}".format(chrom, gene), chrom, beta1, z_score, p_value_two/2)])
        df_cal_zscore = df_cal_zscore.append([("{}.{}".format(chrom, gene), chrom, z_score)])
        ### gene annotation needs to be determined
        if multi_cat == 0:
            SNP_anno_cal = SNP_anno_cal.append([("{}.{}".format(chrom, gene), 1 if gene in causal else 0)])
        else:
            if len(cat) != 0:
                rand_cat = np.random.choice(cat)
            SNP_anno_cal = SNP_anno_cal.append([("{}.{}".format(chrom, gene), 1 if gene in causal else rand_cat)])
            if not int(gene[4:]) in causal:
                cat.remove(rand_cat)
        i += 1
    df_cal_bf.columns = ["SNP", "gene", "bf"]
    df_cal_beta.columns = ["SNP", "gene", "beta", "t-stat", "p-value"]
    df_cal_zscore.columns = ["SNP", "gene", "z-score"]
    SNP_anno_cal.columns = ["SNP", "binding_d"]
    df_cal_bf.to_csv("data/TORUS/calcium/bf_cal", sep = "\t", header = False, index = False)
    df_cal_beta.to_csv("data/TORUS/calcium/beta_cal", sep = "\t", header = True, index = False)
    df_cal_zscore.to_csv("data/TORUS/calcium/z_score_cal", sep = "\t", header = False, index = False)
    SNP_anno_cal.to_csv("data/TORUS/calcium/snp_anno_cal", sep = "\t", header = True, index = False)
#     return df_cal_bf, df_cal_beta, df_cal_zscore, SNP_anno_cal

In [28]:
# there are 0~1847 blocks, block #776 has the most genes involved
df_cal = data["data"][0]
print (len(list(df_cal)))
block_genes = df_cal.columns.values[1:].tolist()
# print ((block_genes))
n_gene = df_cal.shape[1]
pthway_genes = cal_pthwy_genes["gene_name"].tolist()
cat6 = []
for gene in block_genes:
    if gene in pthway_genes:
        cat6.append(1)
    else: cat6.append(0)
# print (len(cat6))

155


In [31]:
print (df_cal)

      phenotype  ADIPOR1  ADORA1  ARL8A  ASCL5  ATP2B4  ATP6V1G3  AVPR1B  \
0             1        0       0      0      0       0         0       0   
1             1        0       0      0      0       0         0       0   
2             1        0       0      0      0       0         0       0   
3             1        0       0      0      0       0         0       0   
4             1        0       0      0      0       0         0       0   
5             1        0       0      0      0       0         0       0   
6             1        0       0      0      0       0         0       0   
7             1        1       1      0      0       1         0       0   
8             1        0       0      0      0       0         0       0   
9             1        0       0      0      0       0         0       0   
10            1        0       0      0      0       0         0       0   
11            1        0       0      0      0       0         0       0   
12          

In [29]:
get_sum_stats_cal_simu(gene_df = df_cal, causal = pthway_genes, cat = cat6, multi_cat = 0, chrom = "chr19")

4 1996 2 1998 2.002004008016032
3 1997 1 1999 3.0030045067601403
3 1997 0 2000 0
13 1987 0 2000 0
7 1993 1 1999 7.021073758153538
3 1997 0 2000 0
6 1994 1 1999 6.015045135406218
3 1997 0 2000 0
4 1996 0 2000 0
8 1992 2 1998 4.0120481927710845
2 1998 3 1997 0.666332999666333
1 1999 2 1998 0.49974987493746875
6 1994 1 1999 6.015045135406218
3 1997 0 2000 0
1 1999 3 1997 0.3329998332499583
1 1999 3 1997 0.3329998332499583
16 1984 2 1998 8.056451612903226
6 1994 3 1997 2.0030090270812435
1 1999 2 1998 0.49974987493746875
1 1999 2 1998 0.49974987493746875
3 1997 1 1999 3.0030045067601403
4 1996 2 1998 2.002004008016032
3 1997 0 2000 0
3 1997 0 2000 0
5 1995 0 2000 0
1 1999 2 1998 0.49974987493746875
0 2000 2 1998 0
2 1998 1 1999 2.001001001001001
6 1994 0 2000 0
7 1993 1 1999 7.021073758153538
4 1996 2 1998 2.002004008016032
5 1995 1 1999 5.010025062656641
4 1996 1 1999 4.006012024048096
5 1995 0 2000 0
4 1996 1 1999 4.006012024048096
4 1996 1 1999 4.006012024048096
2 1998 2 1998 1.0
4 1996

In [30]:
!rm data/TORUS/calcium/*.gz
!gzip data/TORUS/calcium/*
!torus.sh -d data/TORUS/calcium/beta_cal.gz -annot data/TORUS/calcium/snp_anno_cal.gz -est > data/TORUS/calcium/cal_block_beta.est

Read in 1 loci, 154 locus-SNP pairs ... 
Loading annotations ... 
Initializing ... 
Starting EM ... 
  Iter          loglik          Intercept    binding.1	
   1            42.005           -5.011    -43.156  
   2            43.052           -5.011    -86.311  
   3            43.052           -5.011   -129.467  


## Run DAP

In [13]:
dap_method = 'dap-g'
multiplier = 10
fileout = fileout + '_multiplier{}.{}'.format(multiplier, dap_method)
# clean it up, very important!
if os.path.isfile(fileout):
    os.remove(fileout)
os.system('rm -f /tmp/F*{dat,prior,grid}')
for idx, df in enumerate(data['data']):
    df["phenotype"] = df.apply(lambda row: y_tran if row["phenotype"]==1.0 else 0, axis=1)
    times = run_dap(df, fileout, cal_pthwy_genes, ref_genes, grid = [(0, beta)], 
                    multiplier = multiplier, ncpu = 4, dry_run = False, dap_method = dap_method)
    print('Job {2}/{3}, elapsed {0:.2f}s prepare & {1:.2f}s DAP'.\
          format(times[1] - times[0], times[2] - times[1], idx + 1, len(data['data'])))

Job 1/1848, elapsed 2.32s prepare & 1382.34s DAP
Job 2/1848, elapsed 0.09s prepare & 1.35s DAP
Job 3/1848, elapsed 0.03s prepare & 0.68s DAP
Job 4/1848, elapsed 0.81s prepare & 1315.79s DAP
Job 5/1848, elapsed 1.43s prepare & 620.19s DAP
Job 6/1848, elapsed 0.33s prepare & 2.46s DAP
Job 7/1848, elapsed 0.15s prepare & 1.26s DAP
Job 8/1848, elapsed 2.83s prepare & 201.48s DAP
Job 9/1848, elapsed 0.51s prepare & 91.10s DAP
Job 10/1848, elapsed 0.41s prepare & 17.29s DAP
Job 11/1848, elapsed 0.95s prepare & 1385.32s DAP
Job 12/1848, elapsed 0.02s prepare & 0.52s DAP
Job 13/1848, elapsed 0.84s prepare & 787.88s DAP
Job 14/1848, elapsed 0.35s prepare & 4.58s DAP
Job 15/1848, elapsed 0.17s prepare & 2.07s DAP
Job 16/1848, elapsed 0.34s prepare & 4.49s DAP
Job 17/1848, elapsed 0.32s prepare & 4.26s DAP
Job 18/1848, elapsed 0.21s prepare & 2.63s DAP
Job 19/1848, elapsed 1.51s prepare & 3869.17s DAP
Job 20/1848, elapsed 0.45s prepare & 3.42s DAP
Job 21/1848, elapsed 0.11s prepare & 1.54s DAP
Jo

Job 173/1848, elapsed 0.03s prepare & 0.66s DAP
Job 174/1848, elapsed 0.03s prepare & 0.66s DAP
Job 175/1848, elapsed 0.04s prepare & 0.82s DAP
Job 176/1848, elapsed 0.15s prepare & 2.17s DAP
Job 177/1848, elapsed 0.16s prepare & 2.19s DAP
Job 178/1848, elapsed 0.03s prepare & 0.67s DAP
Job 179/1848, elapsed 0.13s prepare & 1.82s DAP
Job 180/1848, elapsed 0.02s prepare & 0.50s DAP
Job 181/1848, elapsed 0.02s prepare & 0.52s DAP
Job 182/1848, elapsed 0.27s prepare & 3.33s DAP
Job 183/1848, elapsed 0.09s prepare & 1.36s DAP
Job 184/1848, elapsed 0.23s prepare & 2.92s DAP
Job 185/1848, elapsed 0.28s prepare & 3.67s DAP
Job 186/1848, elapsed 0.10s prepare & 1.51s DAP
Job 187/1848, elapsed 0.02s prepare & 0.51s DAP
Job 188/1848, elapsed 0.07s prepare & 1.19s DAP
Job 189/1848, elapsed 0.21s prepare & 2.87s DAP
Job 190/1848, elapsed 0.05s prepare & 0.86s DAP
Job 191/1848, elapsed 0.02s prepare & 0.51s DAP
Job 192/1848, elapsed 0.04s prepare & 0.85s DAP
Job 193/1848, elapsed 0.02s prepare & 0.

Job 344/1848, elapsed 0.02s prepare & 0.52s DAP
Job 345/1848, elapsed 0.03s prepare & 0.68s DAP
Job 346/1848, elapsed 0.02s prepare & 0.51s DAP
Job 347/1848, elapsed 0.02s prepare & 0.51s DAP
Job 348/1848, elapsed 0.05s prepare & 0.84s DAP
Job 349/1848, elapsed 0.02s prepare & 0.51s DAP
Job 350/1848, elapsed 0.13s prepare & 1.85s DAP
Job 351/1848, elapsed 0.03s prepare & 0.67s DAP
Job 352/1848, elapsed 0.02s prepare & 0.52s DAP
Job 353/1848, elapsed 0.05s prepare & 0.68s DAP
Job 354/1848, elapsed 0.07s prepare & 1.15s DAP
Job 355/1848, elapsed 0.05s prepare & 0.85s DAP
Job 356/1848, elapsed 0.05s prepare & 0.84s DAP
Job 357/1848, elapsed 0.15s prepare & 2.03s DAP
Job 358/1848, elapsed 0.02s prepare & 0.52s DAP
Job 359/1848, elapsed 0.06s prepare & 1.01s DAP
Job 360/1848, elapsed 0.04s prepare & 0.67s DAP
Job 361/1848, elapsed 0.02s prepare & 0.52s DAP
Job 362/1848, elapsed 0.02s prepare & 0.51s DAP
Job 363/1848, elapsed 0.02s prepare & 0.51s DAP
Job 364/1848, elapsed 1.01s prepare & 29

Job 514/1848, elapsed 0.45s prepare & 3.70s DAP
Job 515/1848, elapsed 0.62s prepare & 105.91s DAP
Job 516/1848, elapsed 1.27s prepare & 1566.14s DAP
Job 517/1848, elapsed 0.53s prepare & 92.53s DAP
Job 518/1848, elapsed 0.95s prepare & 1338.93s DAP
Job 519/1848, elapsed 0.11s prepare & 1.52s DAP
Job 520/1848, elapsed 0.03s prepare & 0.69s DAP
Job 521/1848, elapsed 0.03s prepare & 0.72s DAP
Job 522/1848, elapsed 0.42s prepare & 23.35s DAP
Job 523/1848, elapsed 0.17s prepare & 2.58s DAP
Job 524/1848, elapsed 0.11s prepare & 1.58s DAP
Job 525/1848, elapsed 0.07s prepare & 1.09s DAP
Job 526/1848, elapsed 0.02s prepare & 0.56s DAP
Job 527/1848, elapsed 0.20s prepare & 2.66s DAP
Job 528/1848, elapsed 0.02s prepare & 0.52s DAP
Job 529/1848, elapsed 0.05s prepare & 0.87s DAP
Job 530/1848, elapsed 0.21s prepare & 1.79s DAP
Job 531/1848, elapsed 0.91s prepare & 52.33s DAP
Job 532/1848, elapsed 0.20s prepare & 1.85s DAP
Job 533/1848, elapsed 0.11s prepare & 1.52s DAP
Job 534/1848, elapsed 0.03s p

Job 684/1848, elapsed 0.03s prepare & 0.70s DAP
Job 685/1848, elapsed 0.24s prepare & 3.17s DAP
Job 686/1848, elapsed 0.03s prepare & 0.68s DAP
Job 687/1848, elapsed 0.10s prepare & 1.49s DAP
Job 688/1848, elapsed 0.03s prepare & 0.68s DAP
Job 689/1848, elapsed 0.20s prepare & 2.67s DAP
Job 690/1848, elapsed 0.09s prepare & 1.34s DAP
Job 691/1848, elapsed 0.12s prepare & 1.67s DAP
Job 692/1848, elapsed 0.14s prepare & 2.00s DAP
Job 693/1848, elapsed 0.06s prepare & 1.01s DAP
Job 694/1848, elapsed 0.10s prepare & 1.52s DAP
Job 695/1848, elapsed 0.19s prepare & 2.66s DAP
Job 696/1848, elapsed 0.11s prepare & 1.68s DAP
Job 697/1848, elapsed 0.28s prepare & 3.67s DAP
Job 698/1848, elapsed 0.32s prepare & 4.16s DAP
Job 699/1848, elapsed 0.18s prepare & 2.50s DAP
Job 700/1848, elapsed 0.32s prepare & 4.16s DAP
Job 701/1848, elapsed 0.20s prepare & 2.67s DAP
Job 702/1848, elapsed 0.14s prepare & 1.84s DAP
Job 703/1848, elapsed 0.02s prepare & 0.53s DAP
Job 704/1848, elapsed 0.04s prepare & 0.

...

Job 1024/1848, elapsed 0.38s prepare & 15.87s DAP
Job 1025/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1026/1848, elapsed 0.47s prepare & 37.44s DAP
Job 1027/1848, elapsed 1.94s prepare & 295.15s DAP
Job 1028/1848, elapsed 0.51s prepare & 86.85s DAP
Job 1029/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1030/1848, elapsed 0.26s prepare & 3.44s DAP
Job 1031/1848, elapsed 0.43s prepare & 42.69s DAP
Job 1032/1848, elapsed 0.42s prepare & 42.20s DAP
Job 1033/1848, elapsed 0.03s prepare & 0.69s DAP
Job 1034/1848, elapsed 0.04s prepare & 0.82s DAP
Job 1035/1848, elapsed 0.40s prepare & 21.70s DAP
Job 1036/1848, elapsed 1.11s prepare & 453.96s DAP
Job 1037/1848, elapsed 0.35s prepare & 4.42s DAP
Job 1038/1848, elapsed 0.14s prepare & 1.93s DAP
Job 1039/1848, elapsed 0.07s prepare & 1.14s DAP
Job 1040/1848, elapsed 0.03s prepare & 0.66s DAP
Job 1041/1848, elapsed 0.31s prepare & 27.47s DAP
Job 1042/1848, elapsed 0.10s prepare & 1.51s DAP
Job 1043/1848, elapsed 0.02s prepare & 0.51s DAP
Job 1044/

Job 1191/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1192/1848, elapsed 0.83s prepare & 309.15s DAP
Job 1193/1848, elapsed 0.31s prepare & 4.09s DAP
Job 1194/1848, elapsed 0.50s prepare & 117.03s DAP
Job 1195/1848, elapsed 0.03s prepare & 0.68s DAP
Job 1196/1848, elapsed 0.70s prepare & 145.08s DAP
Job 1197/1848, elapsed 0.20s prepare & 13.14s DAP
Job 1198/1848, elapsed 0.31s prepare & 4.16s DAP
Job 1199/1848, elapsed 0.07s prepare & 1.86s DAP
Job 1200/1848, elapsed 0.21s prepare & 2.90s DAP
Job 1201/1848, elapsed 0.06s prepare & 1.01s DAP
Job 1202/1848, elapsed 0.11s prepare & 2.86s DAP
Job 1203/1848, elapsed 0.26s prepare & 3.32s DAP
Job 1204/1848, elapsed 0.19s prepare & 2.56s DAP
Job 1205/1848, elapsed 0.43s prepare & 36.34s DAP
Job 1206/1848, elapsed 0.08s prepare & 1.32s DAP
Job 1207/1848, elapsed 0.04s prepare & 0.83s DAP
Job 1208/1848, elapsed 0.27s prepare & 21.44s DAP
Job 1209/1848, elapsed 0.27s prepare & 3.66s DAP
Job 1210/1848, elapsed 0.14s prepare & 1.97s DAP
Job 1211/18

Job 1358/1848, elapsed 0.08s prepare & 1.31s DAP
Job 1359/1848, elapsed 0.20s prepare & 2.84s DAP
Job 1360/1848, elapsed 0.07s prepare & 1.16s DAP
Job 1361/1848, elapsed 0.43s prepare & 42.73s DAP
Job 1362/1848, elapsed 0.22s prepare & 1.80s DAP
Job 1363/1848, elapsed 0.05s prepare & 0.68s DAP
Job 1364/1848, elapsed 0.06s prepare & 0.71s DAP
Job 1365/1848, elapsed 0.06s prepare & 0.67s DAP
Job 1366/1848, elapsed 0.20s prepare & 1.68s DAP
Job 1367/1848, elapsed 0.03s prepare & 0.51s DAP
Job 1368/1848, elapsed 0.21s prepare & 2.80s DAP
Job 1369/1848, elapsed 0.18s prepare & 2.45s DAP
Job 1370/1848, elapsed 0.27s prepare & 3.67s DAP
Job 1371/1848, elapsed 0.07s prepare & 1.16s DAP
Job 1372/1848, elapsed 0.02s prepare & 0.51s DAP
Job 1373/1848, elapsed 0.02s prepare & 0.50s DAP
Job 1374/1848, elapsed 0.11s prepare & 1.67s DAP
Job 1375/1848, elapsed 0.03s prepare & 0.65s DAP
Job 1376/1848, elapsed 0.03s prepare & 0.69s DAP
Job 1377/1848, elapsed 0.02s prepare & 0.50s DAP
Job 1378/1848, elap

Job 1525/1848, elapsed 0.19s prepare & 2.45s DAP
Job 1526/1848, elapsed 0.03s prepare & 0.68s DAP
Job 1527/1848, elapsed 0.09s prepare & 1.32s DAP
Job 1528/1848, elapsed 0.06s prepare & 1.03s DAP
Job 1529/1848, elapsed 0.03s prepare & 0.69s DAP
Job 1530/1848, elapsed 0.02s prepare & 0.51s DAP
Job 1531/1848, elapsed 0.03s prepare & 0.69s DAP
Job 1532/1848, elapsed 0.09s prepare & 1.32s DAP
Job 1533/1848, elapsed 0.21s prepare & 2.66s DAP
Job 1534/1848, elapsed 0.22s prepare & 2.98s DAP
Job 1535/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1536/1848, elapsed 0.06s prepare & 0.99s DAP
Job 1537/1848, elapsed 0.10s prepare & 1.46s DAP
Job 1538/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1539/1848, elapsed 0.03s prepare & 0.68s DAP
Job 1540/1848, elapsed 0.02s prepare & 0.51s DAP
Job 1541/1848, elapsed 0.02s prepare & 0.51s DAP
Job 1542/1848, elapsed 0.02s prepare & 0.50s DAP
Job 1543/1848, elapsed 0.03s prepare & 0.66s DAP
Job 1544/1848, elapsed 0.03s prepare & 0.67s DAP
Job 1545/1848, elaps

Job 1692/1848, elapsed 0.27s prepare & 2.22s DAP
Job 1693/1848, elapsed 0.16s prepare & 1.40s DAP
Job 1694/1848, elapsed 0.02s prepare & 0.54s DAP
Job 1695/1848, elapsed 0.14s prepare & 1.97s DAP
Job 1696/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1697/1848, elapsed 0.19s prepare & 2.52s DAP
Job 1698/1848, elapsed 0.03s prepare & 0.69s DAP
Job 1699/1848, elapsed 0.03s prepare & 0.68s DAP
Job 1700/1848, elapsed 0.02s prepare & 0.54s DAP
Job 1701/1848, elapsed 0.10s prepare & 1.52s DAP
Job 1702/1848, elapsed 0.05s prepare & 0.85s DAP
Job 1703/1848, elapsed 0.03s prepare & 0.69s DAP
Job 1704/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1705/1848, elapsed 0.24s prepare & 3.09s DAP
Job 1706/1848, elapsed 0.06s prepare & 1.02s DAP
Job 1707/1848, elapsed 0.07s prepare & 1.17s DAP
Job 1708/1848, elapsed 0.03s prepare & 0.68s DAP
Job 1709/1848, elapsed 0.12s prepare & 1.87s DAP
Job 1710/1848, elapsed 0.06s prepare & 1.03s DAP
Job 1711/1848, elapsed 0.02s prepare & 0.52s DAP
Job 1712/1848, elaps

In [1]:
# dap/dap -d /tmp/F1503014064.6944265.dat -g /tmp/F1503014064.6944265.grid -it 0.05 -prior /tmp/F1503014064.6944265.prior -t 2 > data/calcium_pathway_N4000_shape25_nomask.data.dap 

In [9]:
# calpath_gene_pos.to_csv("test.csv", sep="\t")
print (cal_pthwy_genes["gene_name"].tolist())

['ADCY1', 'ADCY2', 'ADCY3', 'ADCY4', 'ADCY7', 'ADCY8', 'ADCY9', 'ADORA2A', 'ADORA2B', 'ADRA1A', 'ADRA1B', 'ADRA1D', 'ADRB1', 'ADRB2', 'ADRB3', 'AGTR1', 'ATP2A1', 'ATP2A2', 'ATP2A3', 'ATP2B1', 'ATP2B2', 'ATP2B3', 'ATP2B4', 'AVPR1A', 'AVPR1B', 'BDKRB1', 'BDKRB2', 'BST1', 'CACNA1A', 'CACNA1B', 'CACNA1C', 'CACNA1D', 'CACNA1E', 'CACNA1F', 'CACNA1G', 'CACNA1H', 'CACNA1I', 'CACNA1S', 'CALM1', 'CALM2', 'CALM3', 'CALML3', 'CALML5', 'CALML6', 'CAMK2A', 'CAMK2B', 'CAMK2D', 'CAMK2G', 'CAMK4', 'CCKAR', 'CCKBR', 'CD38', 'CHP', 'CHP2', 'CHRM1', 'CHRM2', 'CHRM3', 'CHRM5', 'CHRNA7', 'CYSLTR1', 'CYSLTR2', 'DRD1', 'DRD5', 'EDNRA', 'EDNRB', 'EGFR', 'ERBB2', 'ERBB3', 'ERBB4', 'F2R', 'GNA11', 'GNA14', 'GNA15', 'GNAL', 'GNAQ', 'GNAS', 'GRIN1', 'GRIN2A', 'GRIN2C', 'GRIN2D', 'GRM1', 'GRM5', 'GRPR', 'HRH1', 'HRH2', 'HTR2A', 'HTR2B', 'HTR2C', 'HTR4', 'HTR5A', 'HTR6', 'HTR7', 'ITPKA', 'ITPKB', 'ITPR1', 'ITPR2', 'ITPR3', 'LHCGR', 'LOC729317', 'LTB4R2', 'MYLK', 'MYLK2', 'MYLK3', 'NOS1', 'NOS2', 'NOS3', 'NTSR1', 'OX

In [ ]:
# chrom
# chrom = ["chr{}".format(i) for i in list(range(1,23))+["X"]]
# print ((chrom))

In [ ]:
# [I 15:34:54.151 NotebookApp] Accepting one-time-token-authenticated connection from ::1
# [I 15:34:54.814 NotebookApp] Kernel started: 60079e22-d81d-4f83-a293-c5750b5456a1
# [I 15:36:54.808 NotebookApp] Saving file at /20170726_Dap_on_simulation.ipynb
# [W 15:42:01.170 NotebookApp] WebSocket ping timeout after 92790 ms.
# [I 15:44:57.820 NotebookApp] KernelRestarter: restarting kernel (1/5)